# NDVI Classification for May

The purpose of this code is to retrieve Sentinel 2 images for Samos Island, Greece, in order to perform a mapping of Psili Ammos, a wetland located in the Southeast region. The code includes the following steps:

- Data Retrieval: Retrieve Sentinel 2 images for the specified area and time range.

- Preprocessing: Apply necessary preprocessing steps such as cloud masking, band selection, and image enhancement   techniques.

- Feature Extraction: Calculate relevant indices such as Normalized Difference Water Index (NDWI) and Soil Adjusted Vegetation Index (SAVI).

- Unsupervised Classification: Perform an unsupervised classification using the extracted features.

- Accuracy Assessment: Evaluate the accuracy of the classification by comparing it with a ground truth dataset created on-site.

By Michel Tarby

# Sentinel-2 Images retrievingn and treatments

### Packages importations 

In [2]:
import importlib
import sys
import subprocess
import os
import earthpy.plot as ep
import pandas as pd


def install_and_import(package):
    try:
        importlib.import_module(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    finally:
        globals()[package] = importlib.import_module(package)

packages = ['ee', 'geemap', 'rasterio', 'rasterstats', 'geopandas', 'numpy', 'scipy', 'matplotlib']

for package in packages:
    install_and_import(package)

# Additional imports
from scipy.optimize import curve_fit
from pandas.tseries.offsets import MonthEnd
import matplotlib.pyplot as plt
import geemap.common as com

In [3]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

Enter verification code: 4/1AZEOvhUAVI-Xe3aiX4AaWqaYuZjoCsmJK_sbWlIgnCzwP8thNqhGPZscDoY

Successfully saved authorization token.


### Map instance with a zoom on Psili Ammos area

In [4]:
# Create a geemap Map instance
Map = geemap.Map()

# Define the coordinates and zoom level for the region of interest (ROI) - (Samos, in this case)
roi_longitude = 27.009654
roi_latitude = 37.707147
zoom_level = 15

# Center the map on the ROI 
Map.setCenter(roi_longitude, roi_latitude, zoom_level)

# Display the map
Map

Map(center=[37.707147, 27.009654], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

### Retrieve Sentinel 2 Images and put shape of Psili Ammos on top

In [5]:
# Function to mask clouds in Sentinel-2 images
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

# Define the study area and date range
study_area = ee.Geometry.Point(roi_longitude, roi_latitude)
start_date = '2023-05-01'
end_date = '2023-05-31'

# Load Sentinel-2 images and apply cloud masking, further we select only imagery with less than 5% cloud

dataset_image = (ee.ImageCollection('COPERNICUS/S2_SR')
                 .filterBounds(study_area)
                 .filterDate(start_date, end_date)
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
                 .map(maskS2clouds))

# Define visualization parameters
visualization = {
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

# Add the Sentinel-2 image layer to the map
Map.addLayer(dataset_image, visualization, 'S2')

# Import delimitations of the wetland
samos_area = ee.FeatureCollection('projects/archipelagos-michel-project/assets/samos_area')
Map.addLayer(samos_area, {'color': 'purple'}, 'Area samos')
feature_geometry = samos_area.geometry() #delimitations to clip NDVI and NDWI

print('Image collection size:', dataset_image.size().getInfo())
sample_image = dataset_image.first()  # Get the first image from the collection
band_names = sample_image.bandNames().getInfo()
print('Sample image band names:', band_names)

Image collection size: 1
Sample image band names: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60']


### Creation of beginning date and ending date for each month for the considerate period

In [6]:
# End and begin dates for median creation
date_range = pd.date_range('2022-01-01', '2022-12-31', freq='MS')

begin_date = date_range.strftime("%Y-%m-%d")
end_date = (date_range + MonthEnd(1)).strftime("%Y-%m-%d")
print(end_date)

Index(['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31',
       '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31',
       '2022-11-30', '2022-12-31'],
      dtype='object')


In [7]:
# Iterate over each month
median_images = []
NDVI_month = []
NDWI_month = []
SWI_month = []
SAVI_month = []

for i in range(0,12):
    month_start = begin_date[i]
    month_end = end_date[i]
    month_images = (ee.ImageCollection('COPERNICUS/S2_SR')
                 .filterBounds(study_area)
                 .filterDate(month_start, month_end)
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
                 .map(maskS2clouds))
    month_median = month_images.median()
    
    # If there is no images with less than 5% cloud, then we try to have other with at max 20%
    if month_median.bandNames().size().eq(0):
        month_images = (ee.ImageCollection('COPERNICUS/S2_SR')
                 .filterBounds(study_area)
                 .filterDate(month_start, month_end)
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60))
                 .map(maskS2clouds))
        month_median = month_images.median()

    # Normalized difference is computed only once
    ndvi = month_median.normalizedDifference(['B8', 'B4'])
    ndwi = month_median.normalizedDifference(['B3', 'B8'])
    swi = month_median.normalizedDifference(['B5', 'B11'])
    savi = month_median.expression(
      '((B8 - B4) / (B8 + B4 + 0.428)) * 1.428',
      {
        'B8': month_median.select('B8'),
        'B4': month_median.select('B4')
      }
    )   

    masked_ndvi = ndvi.clip(feature_geometry)
    masked_ndwi = ndwi.clip(feature_geometry)
    masked_savi = savi.clip(feature_geometry)
    masked_swi = swi.clip(feature_geometry)
    
    median_images.append(month_median)
    NDVI_month.append(masked_ndvi)
    NDWI_month.append(masked_ndwi)
    SAVI_month.append(masked_savi)
    SWI_month.append(masked_swi)


In [12]:
# Select the NDVI image for a specific month (e.g., month 4)
ndvi_image = NDVI_month[0]

# Define visualization parameters for the NDVI
ndvi_vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
}

# Add the NDVI image to the GEE map
Map.addLayer(ndvi_image, ndvi_vis_params, 'NDVI Image')
Map

Map(bottom=3244692.0, center=[37.706368356809776, 27.009800152221047], controls=(WidgetControl(options=['posit…

In [13]:
# Import ground truth points on the map
sampling_points = ee.FeatureCollection('projects/archipelagos-michel-project/assets/Data_wetland_1106')
Map.addLayer(sampling_points, {'color': 'red'}, 'Ground truth points')


# Unsupervised classification NDWI and then SAVI

## NDWI supervised classification

In [15]:
# Import ground truth points
sampling_points = ee.FeatureCollection('projects/archipelagos-michel-project/assets/Data_wetland_1106')

# Replace other than Water (ClassId = 0) by Other (ClassId = 1)
def replaceNonZero(feature):
    classId = feature.getNumber('ClassId')
    newClassId = ee.Algorithms.If(classId.neq(0), 1, classId)
    return feature.set('ClassId', newClassId)

# Apply the function to the feature
NDWI_gt = sampling_points.map(replaceNonZero)

In [16]:
# Import Ground truth data
features = NDWI_gt
# Define the desired percentage range for training data
target_percentage_min = 0.65
target_percentage_max = 0.75

# Initialize variables
seed = 0
training_percentage = 0
validation_percentage = 1

# Define the percentage value for splitting
percentage = 0.7  
    
# Iterate until the desired percentage range is achieved
while (
    training_percentage < target_percentage_min or
    training_percentage > target_percentage_max
):
    # Update the seed value
    seed += 1
    feature_random = features.randomColumn('random', seed)

    
    # Split the feature collection into training and validation datasets
    training = feature_random.filter(ee.Filter.lte('random', percentage))
    validation = feature_random.filter(ee.Filter.gt('random', percentage))

    # Calculate the current training and validation percentages
    total_count = training.size().add(validation.size())
    training_percentage = training.size().divide(total_count).getInfo()

validation_percentage = validation.size().divide(total_count).getInfo()

# Print the number of features in each dataset
print('Number of training features:', training.size().getInfo())
print('Number of validation features:', validation.size().getInfo())
print('Percentage of training features:', training_percentage * 100, '%')
print('Seed number:', seed)


Number of training features: 69
Number of validation features: 24
Percentage of training features: 74.19354838709677 %
Seed number: 2


In [17]:
#Load May 2023 Image
may2023 = (ee.ImageCollection('COPERNICUS/S2_SR')
                 .filterBounds(study_area)
                 .filterDate('2023-05-01', '2023-05-31')
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
                 .map(maskS2clouds))
may2023_median = may2023.median()

# Training with May 2023 data
image_2023 = may2023_median.normalizedDifference(['B3', 'B8'])

#Load NDWI values for each point
trainingData_2023 = image_2023.sampleRegions(collection= training, properties= ['ClassId'],scale= 10)
validationData_2023 = image_2023.sampleRegions(collection= validation, properties= ['ClassId'],scale= 10)

#Train a Random Forest classifier
classifier_2023 = ee.Classifier.smileRandomForest(10).train(trainingData_2023, 'ClassId', ['nd'])
classified_2023 = image_2023.classify(classifier_2023)

#Classify validation dataset
validationClassification_2023 = validationData_2023.classify(classifier_2023)

# Set the visualization parameters
vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['blue','grey']
}

# Add classified 2023 image layer
Map.addLayer(classified_2023, vis_params, 'Classif May 2023')
Map

Map(bottom=3244692.0, center=[37.706368356809776, 27.009800152221047], controls=(WidgetControl(options=['posit…

In [18]:
# Compute Error matrix on validation dataset
def createMatrix_supervised(data):
    trainAccuracy = data.errorMatrix("ClassId", "classification")
    print('Resubstitution error matrix: ', trainAccuracy.getInfo())
    print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())
    
# Error matrix construction
createMatrix_supervised(validationClassification_2023)

Resubstitution error matrix:  [[7, 0], [1, 16]]
Training overall accuracy:  0.9583333333333334


### Apply the classifier for all months of the considered year

In [20]:
# Import Psili Ammos geometry
ps_am = ee.FeatureCollection('projects/archipelagos-michel-project/assets/ps_am')
# Initialize variable and lists
classified_images=[]
land_mask=[]

# Loop for each NDWI image
for month in NDWI_month:
    # Apply classifier for each image
    input = month
    classified = input.classify(classifier_2023)
    classified_images.append(classified)
    land_mask.append(classified.gt(0))


In [15]:
# Display classification for september (water extent ~ 0)
Map.addLayer(classified_images[8], vis_params, 'September 2022 - NDWI')

In [ ]:
# Cross validation for supervised classification (NDWI)
from sklearn.model_selection import cross_val_score


## SAVI unsupervised classification

In [23]:
# Apply mask for considerate month, here May
SAVI_wo_water=[]
for i in range(12):
    SAVI_wo_water.append(SAVI_month[i].updateMask(land_mask[i]))

#SAVI_May_wo_water = SAVI_month[4].updateMask(land_mask)
Map.addLayer(SAVI_wo_water[8],ndvi_vis_params,'SAVI w/o water - September')

# SAVI in 2023 image
image_2023_SAVI = may2023_median.expression(
      '((B8 - B4) / (B8 + B4 + 0.428)) * 1.428',
      {
        'B8': month_median.select('B8'),
        'B4': month_median.select('B4')
      }
    ).updateMask(classified_2023.gt(0))

In [24]:
SAVI_cluster=[]
for month in SAVI_wo_water:
    # Iterate in images 
    input = month

    #training region is the full image
    training = input.sample(region = samos_area, scale = 10)

    #train cluster on image
    clusterer = ee.Clusterer.wekaKMeans(3).train(training)

    #cluster the complete image
    SAVI_cluster.append(input.cluster(clusterer))

In [25]:
#Display the clusters with labels
legend_keys = ['Cluster One', 'Cluster Two', 'Cluster Three', 'Cluster Four']
legend_colors = ['#8DD3C7', '#FFFFB3' ,'#BEBADA']

Map.addLayer(SAVI_cluster[0], {'min': 0, 'max': 2, 'palette': legend_colors}, 'clusters')

In [53]:
# Apply unsupervised classification to may 2023

training2023 = image_2023_SAVI.sample(region = samos_area, scale = 10)

clusterer = ee.Clusterer.wekaKMeans(3).train(training2023)

classif2023 = image_2023_SAVI.cluster(clusterer)

# Remap to have right value for each class
result_remap_SAVI2023 = classif2023.remap([0, 1,2], [2,0, 1])

legend_colors = ['grey', 'yellow', 'green']

Map.addLayer(result_remap_SAVI2023, {'min': 0, 'max': 2, 'palette': legend_colors}, 'Labelled clusters remap 2023')

### Accuracy assessment for unsupervised classification SAVI

In [44]:
# Remap to have right value for each class
result_remap_SAVI = SAVI_cluster[4].remap([0, 1,2], [1, 2,0])

legend_colors = ['grey', 'yellow', 'green']

Map.addLayer(result_remap_SAVI, {'min': 0, 'max': 2, 'palette': legend_colors}, 'Labelled clusters remap')
#Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')

In [64]:
#Export the result
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'SAVI_classif_September2022test.tif')

# Set the value for out of the classification (Water)
SAVI_classif = (result_remap_SAVI.unmask(3))

fixed_image = SAVI_classif.set("nodata_value", -99)

# Export the image to GeoTIFF 
geemap.ee_export_image(
    fixed_image,
    filename=out_file,
    scale=10)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\HP OMEN\Downloads\SAVI_classif_September2022test.tif


In [27]:
# Compute Error matrix 
def createMatrix_unsupervised(data):
    trainAccuracy = data.errorMatrix("ClassId", "remapped")
    print('Resubstitution error matrix: ', trainAccuracy.getInfo())
    print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())


In [55]:
# Refactorize points for matching values of the map
def refactorize_gt(feature):
    classId = feature.getNumber('ClassId')
    newClassId = ee.Algorithms.If(classId.eq(4), 0,ee.Algorithms.If(classId.eq(3),2,classId)) 
    return feature.set('ClassId', newClassId)

# Refactorize ground-truth points 
SAVI_gt = sampling_points.map(refactorize_gt)

# Getting pixel values
g_truth_SAVI = result_remap_SAVI2023.sampleRegions(collection= SAVI_gt
                                               , properties= ['ClassId'],scale= 10)

# Generating error Matrix
createMatrix_unsupervised(g_truth_SAVI)

Resubstitution error matrix:  [[7, 3, 1], [3, 8, 3], [2, 9, 34]]
Training overall accuracy:  0.7


# DRAFT - BEGIN

In [81]:
#test
swi_test = dataset_image.median()
swi_test = swi_test.normalizedDifference(['B3', 'B8'])

# Create a dictionary to map the old band names to the new ones
band_name_mapping = {
    'nd': 'B8'
}

# Update the band names using the select function and the mapping dictionary
#swi_test = swi_test.select(list(band_name_mapping.keys()), list(band_name_mapping.values()))

#Load NDVI values for each point
trainingData_swi = swi_test.sampleRegions(collection= NDWI_gt, properties= ['ClassId'],scale= 10)
validationData_swi = swi_test.sampleRegions(collection= NDWI_gt, properties= ['ClassId'],scale= 10)

#Train a Random Forest classifier
classifier_swi = ee.Classifier.smileRandomForest(10).train(trainingData_swi, 'ClassId', ['nd'])
classified_swi = swi_test.classify(classifier_swi)

#Classify validation dataset
validationClassification_swi = validationData_swi.classify(classifier_swi)
print(swi_test.bandNames().getInfo())

Map.addLayer(classified_swi, vis_params, 'testclassif swi')
Map

['nd']


Map(bottom=3244669.025331096, center=[37.70714840504666, 27.011651965066072], controls=(WidgetControl(options=…

In [272]:
ah = NDWI_month[0].classify(classifier_swi)
Map.addLayer(ah, vis_params, 'ah')

In [194]:
# Define the palette colors
palette = ['blue','grey']  # Blue, Yellow, Light Green, Green

# Set the visualization parameters
vis_params = {
    'min': -1,
    'max': 1,
    'palette': palette
}
Map.addLayer(classified, vis_params, 'testclassif')
Map

Map(bottom=3244695.0, center=[37.706266498600485, 27.009588747467784], controls=(WidgetControl(options=['posit…

In [26]:
#Error matrix and overall accuracy
def createMatrix(data):
    trainAccuracy = data.errorMatrix("ClassId", "classification")
    print('Resubstitution error matrix: ', trainAccuracy.getInfo())
    print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

createMatrix(validationClassification)


NameError: name 'validationClassification' is not defined

# DRAFT - END